<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
    </a>
</p>

**<h1>Capstone</h1>**

**<h1>SpaceX Falcon 9 First Stage Landing Prediction</h1>**

# **Visual Data Analysis**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, ie: the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

<h2>Objectives<h2>

* **Task 1:** Mark all launch sites on a map
* **Task 2:** Mark the success/failed launches for each site on the map
* **Task 3:** Calculate the distances between a launch site to its proximities

## <h2><u>Steps</u></h2>

### 1. Import Required Libraries

In [1]:
import folium
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

### 2. Load the SpaceX Dataset into the Dataframe

[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)

In [2]:
# Load the SpaceX Dataset into a DataFrame
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

In [3]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


<h3>Coordinates for Each Site</h3>

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

In [5]:
# Create a DataFrame containing launch sites and their coordinates
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

In [6]:
# Display
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

In [7]:
# Create a folium Map centered at NASA Johnson Space Center, Houston, Texas
nasa_coordinate = [29.559684888503615, -95.0830971930759]

In [8]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a marker for NASA Johnson Space Center with a text label
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(40,40),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:blue;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

In [10]:
# Add the circle and marker to the map
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

## Tasks

### 1. Launch Sites

Mark all launch sites on a map.

In [11]:
## Task 1: Mark all launch sites on the map

# Add a circle and marker for each launch site
for index, row in launch_sites_df.iterrows():  # `.iterrows()`: Function to iterate over rows in a DataFrame.
    # Create a circle for the launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,  # Radius of the circle
        color='#d35400',  # Circle color
        fill=True,
        fill_color='#d35400'
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)

    # Create a marker with a text label for the launch site
    folium.map.Marker(
        [row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    ).add_to(site_map)

In [12]:
# Display the map
site_map

#### <u>Insights</u>

1. Proximity to the Equator:
  * Context: Latitude of each launch site:
    * CCAFS LC-40: Latitude = 28.562302°
    * VAFB SLC-4E: Latitude = 34.632834°
    * KSC LC-39A: Latitude = 28.573255°
    * CCAFS SLC-40: Latitude = 28.563197°
  * Findings:
    * The launch sites at CCAFS LC-40, KSC LC-39A, and CCAFS SLC-40 are relatively close to the Equator, with latitudes around 28 degrees. These sites are located in Florida, USA, which is relatively closer to the Equator compared to many other regions.
    * However, VAFB SLC-4E is located at a latitude of 34.63 degrees, which is farther north and significantly farther from the Equator compared to the Florida sites.
  * Conclusion:
    * Not all SpaceX launch sites are close to the Equator. Sites in Florida are relatively close, but Vandenberg in California is much farther north.

2. Proximity to the Coast:
  * Findings:
    * CCAFS LC-40 and CCAFS SLC-40: Both of these launch sites are located at Cape Canaveral, Florida, right on the Atlantic coast. They are in very close proximity to the coast.
    * KSC LC-39A: This launch site is part of the Kennedy Space Center, also located on Merritt Island on the Florida coast, very close to the Atlantic Ocean.
    * VAFB SLC-4E: This launch site is located at Vandenberg Air Force Base in California, which is also very close to the Pacific Ocean.
  * Conclusion:
    * All launch sites are located very close to the coast, which is a common requirement for safe rocket launches to minimize risks to populated areas.

### 2. Success/ Failed Launches

Mark the success/failed launches for each site on the map.

Enhance the map by adding the launch outcomes for each site. Recall that Dataframe `spacex_df` has Launch Records, and the `class` column indicates if this launch was successful or not.

In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Create markers for all Launch Records.

If a launch was successful `(class=1)`, use a green marker and if a launch was failed, use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object.

In [14]:
### Task 2: Mark the success/failed launches for each site on the map

# Create a new column 'marker_color' in spacex_df to store the marker color based on launch success
def assign_marker_color(launch_outcome):
    return 'green' if launch_outcome == 1 else 'red'

# Apply the marker color function to the DataFrame
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

In [15]:
# Display
spacex_df.tail()

,Launch Site,Lat,Long,class,marker_color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


In [16]:
# Initialize a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)  # `MarkerCluster()`: Clusters (groups) multiple markers at similar locations.

In [17]:
# Iterate over each row in the spacex_df DataFrame and add markers to the marker_cluster
for index, record in spacex_df.iterrows():
    # Create a marker for each launch, with a color indicating success (green) or failure (red)
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        # 'color' is for the marker background, 'icon_color' is for the marker icon (the inner part)
        icon=folium.Icon(color=record['marker_color']),  # color sets the marker background
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )

In [18]:
# Add the MarkerCluster to the site_map
site_map.add_child(marker_cluster)

# Display the map with all launch sites and markers
site_map

### 3. Proximities

Calculate the distances between a Launch Site to its proximities.

Add a `MousePosition` on the map to get coordinate for a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway).

In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

Zoom in to a Launch Site and explore its proximity to easily find any railway, highway, coastline, etc.
Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [20]:
# Function to calculate distance between two geographical points using Haversine formula

from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

#### 3. 1. Coastline

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [21]:
# Example: Using CCAFS LC-40 as a reference launch site
launch_site_lat = 28.562302
launch_site_lon = -80.577356

In [22]:
# Coordinates for the closest coastline (you can get this using the MousePosition tool on the map)
coastline_lat = 28.56344
coastline_lon = -80.56787

In [23]:
# Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print("Distance to Coastline: {:.2f} KM".format(distance_coastline))

Distance to Coastline: 0.94 KM


In [24]:
# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    icon=folium.Icon(color='blue', icon='info-sign'),
    popup='Launch Site'
).add_to(site_map)

In [25]:
# Add a marker for the closest coastline point
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:blue;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
).add_to(site_map)

In [26]:
# Draw a PolyLine between the launch site and the coastline
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')
site_map.add_child(lines)

#### 3. 2. Railway

Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use MousePosition to find the their coordinates on the map first.

In [27]:
# Coordinates for Railway
railway_lat = 28.5715
railway_lon = -80.58527

In [28]:
# Calculate the distance between the Launch Site and the Railway
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
print("Distance to Railway: {:.2f} KM".format(distance_railway))

Distance to Railway: 1.28 KM


In [29]:
# Add a marker for the Railway
marker_railway = folium.Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(  # `DivIcon`: Display custom HTML content (like text or labels) at a marker's location on the map.
        icon_size=(20,20),
        icon_anchor=(0,0),  # `(0,0)`: Means the top-left corner of the icon is anchored to the marker's exact location (latitude, longitude).
        html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    )
).add_to(site_map)

In [30]:
# Draw a PolyLine between the Launch Site and the Railway
coordinates = [[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]]
lines = folium.PolyLine(locations=coordinates, weight=2, color='red')
site_map.add_child(lines)

#### 3. 3. Highway

In [31]:
# Coordinates for the Highway
highway_lat = 28.56257
highway_lon = -80.57066

In [32]:
# Distance between the Launch Site and the Highway
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
print("Distance to Highway: {:.2f} KM".format(distance_highway))

Distance to Highway: 0.65 KM


In [33]:
# Marker for the Highway
marker_highway = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:grey;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
).add_to(site_map)

In [34]:
# PolyLine between the Launch Site and the Highway
coordinates = [[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]]
lines = folium.PolyLine(locations=coordinates, weight=2, color='grey')
site_map.add_child(lines)

#### 3. 4. City

In [35]:
# Coordinates for the City
city_lat = 28.38566
city_lon = -80.60429

In [36]:
# Distance between the Launch Site and the City
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
print("Distance to City: {:.2f} KM".format(distance_city))

Distance to City: 19.82 KM


In [37]:
# Marker for the City
marker_city = folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:green;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
).add_to(site_map)

In [38]:
# PolyLine between the Launch Site and the City
coordinates = [[launch_site_lat, launch_site_lon], [city_lat, city_lon]]
lines = folium.PolyLine(locations=coordinates, weight=2, color='green')
site_map.add_child(lines)

#### <u>Insights</u>

1. Launch Sites in close proximity to Coastlines: Yes.
2. Launch Sites in close proximity to Railways: Yes
3. Launch Sites in close proximity to Highways: Yes; little farther.
4. Launch Sites keep certain distance away from Cities: Yes.